In [3]:
import os
from pyuvdata import UVData
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import numpy.ma as ma

In [2]:
uv = UVData()
uv.read('/lustre/aoc/projects/hera/pstar/autos/2458745_autos_sum.uvh5')

In [4]:
print(uv.antenna_numbers)

[  0   1   2  11  12  13  14  23  24  25  26  39  84  85  86  87  89  91
 101 102 103 104 105 106 108 120 121 122 123 124 125 126]


In [6]:
freq = uv.freq_array[0]*1e-6
antnums = uv.antenna_numbers
times = np.unique(uv.time_array)